In [5]:
import numpy as np
import pandas as pd

import math
import matplotlib.pyplot as plt
import prosail

from joblib import Parallel, delayed
from tqdm import tqdm
from pyDOE import lhs

In [6]:
nmin, nmax = 1, 3
cabmin, cabmax = 1, 100
laimin, laimax = 0.4, 5
carmin, carmax = 1, 15
cbrownmin, cbrownmax = 0, 0.1
cwmin, cwmax = 0, 0.02
cmmin, cmmax = 0, 0.02
ttsmin,ttsmax=0,70
ttomin,ttomax=0,70
protmin, protmax = 0.001, 0.003
cbcmin, cbcmax = 0, 0.01
antmin, antmax = 0, 10

x = {'n': np.arange(nmin, nmax, 0.001),
     'cab': np.arange(cabmin, cabmax, 0.001),
     'lai': np.arange(laimin, laimax, 0.1),
     'car': np.arange(carmin, carmax, 0.001),
     'cbrown': np.arange(cbrownmin, cbrownmax, 0.001),
     'cw': np.arange(cwmin, cwmax, 0.0001),
     'cm': np.arange(cmmin, cmmax, 0.0001),
     'prot': np.arange(protmin, protmax, 0.0001),
     'tts': np.arange (ttsmin,ttsmax,1),
     'tto': np.arange (ttomin,ttomax,1),
     'cbc': np.arange(cbcmin, cbcmax, 0.0001),
     'ant': np.arange(antmin, antmax, 0.0001)}
num_samples = 5000  # Number of samples in the Latin hypercube

# Generate the Latin hypercube samples
lhs_samples = lhs(len(x), samples=num_samples)

# Scale the Latin hypercube samples to the given parameter ranges
for i, key in enumerate(x):
    lhs_samples[:, i] = x[key][0] + lhs_samples[:, i] * (x[key][-1] - x[key][0])

# Create a DataFrame with the Latin hypercube samples
df3 = pd.DataFrame(lhs_samples, columns=x.keys())

In [7]:
mylist=[]
# for i in tqdm(range(num_samples)):
def process_sample(i, df3):
#     x1 = df3.iloc[i,:]
    # ... rest of the code that processes the sample ...
    
    x1=df3.iloc[i,:]
    rho= prosail.run_prosail(n= x1['n'],alpha=90,cab=x1['cab'],car=x1['car'],cbrown =x1['cbrown'] ,
                                        cw=x1['cw'],cm=x1['cm'],prot=x1['prot'], cbc=x1['cbc'],
                             ant=x1['ant'],prospect_version='PRO',lai=x1['lai'], 
                             lidfa=1, hspot=1, tts=x1['tts'], tto=x1['tto'],psi=0,
                             rsoil0=np.zeros(2101), rsoil=None, psoil=None)
                            
    ##################################################################################################################################################	
    ### USER DEFINED MODEL INPUTS ###
    ##################################################################################################################################################	
    spectral = {}
    leafbio = {}
    opticalParameters = {}

    # Biochemical inputs #
    leafbio['Cab'] = x1['cab']  # Chlorophyll content [ug cm^-2]
    leafbio['Cca'] = x1['car']  # Carotenoid content [ug cm^-2]
    leafbio['Cs'] = x1['cbrown']# Fraction of senescent matter [0-1]
    leafbio['Cw'] = x1['cw']  # Water column [cm]
    leafbio['Cdm'] = x1['cm']  # Dry matter content [g cm^-2] # If Cdm = 0 then Cbc and Cp are used instead
    leafbio['N'] = x1['n']  # Messophyl structural parameter [1.0-3.5]
    leafbio['fqe'] = 0.4  # Chl. fluorescence quantum yield combined for both PSI and PSII, default = .025
                                    # Chl. fluorescence is not simulated if leafbio.fqe = 0
    leafbio['V2Z'] = 1  # Violaxanthin-Zeaxanthin deepoxidation status [0-1]; 0 ~100# Violaxanthin, 1 ~100# Zeaxanthin
                                    # Violaxanthin-Zeaxanthin deepoxidation is not simulated if leafbio.V2Z = -999 
    leafbio['Can'] = x1['ant']  # Anthocyanin content [ug cm^-2]; Anthocyanins are not simulated if Can = 0
    leafbio['Cbc'] = x1['cbc']  # Carbon-based dry matter constituents' content [g.cm-2] #  Simulated only if Cdm = 0
    leafbio['Cp'] = x1['prot']  # Nitrogen-based dry matter protein content [g.cm-2] # Simulated only if Cdm = 0
    leafbio['lai'] = x1['lai']

    dfdf=pd.DataFrame(rho).T
    df = pd.DataFrame(leafbio.values(), index=leafbio.keys()).T
    waves=list(range(400,2501))
    bionames=['Cab', 'Cca', 'Cs', 'Cw', 'Cdm', 'N', 'fqe', 'V2Z', 'Can', 'Cbc', 'Cp','lai']
    bands=np.array(dfdf).flatten()
    bio=np.array(df).flatten()
    dfdf=pd.DataFrame(bands).T
    dfdf.columns=waves
    bio=pd.DataFrame(bio).T
    bio.columns=bionames
    concatenated_df = pd.concat([dfdf, bio],axis=1)
    # concatenated_df["SIF"]=sif_far_red_peak
#     mylist.append(concatenated_df)
    return concatenated_df


In [8]:
num_samples = len(df3)
results = Parallel(n_jobs=-1)(delayed(process_sample)(i, df3) for i in tqdm(range(num_samples)))

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:25<00:00, 197.32it/s]


In [10]:
temp=pd.concat(results,axis=0)


In [11]:
temp1=temp.iloc[:,0:2101]
temp1.shape

(5000, 2101)

In [14]:
temp1

,400,401,402,403,404,405,406,407,408,409,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,0.050962,0.050970,0.050979,0.050989,0.051003,0.051019,0.050982,0.050947,0.050921,0.050887,...,0.072073,0.072087,0.072128,0.072261,0.072320,0.072548,0.072435,0.072802,0.072485,0.073030
0,0.080946,0.080948,0.080950,0.080952,0.080955,0.080960,0.080876,0.080793,0.080722,0.080639,...,0.074211,0.074088,0.074033,0.073989,0.073978,0.074068,0.073910,0.074061,0.073814,0.074063
0,0.086911,0.086918,0.086925,0.086934,0.086947,0.086963,0.086887,0.086813,0.086754,0.086681,...,0.086058,0.085956,0.085917,0.085920,0.085930,0.086081,0.085925,0.086179,0.085895,0.086297
0,0.072528,0.072528,0.072528,0.072529,0.072529,0.072530,0.072452,0.072374,0.072307,0.072229,...,0.049262,0.049215,0.049189,0.049160,0.049155,0.049178,0.049135,0.049165,0.049113,0.049162
0,0.078534,0.078533,0.078533,0.078533,0.078535,0.078539,0.078457,0.078376,0.078307,0.078226,...,0.100756,0.100533,0.100440,0.100360,0.100330,0.100467,0.100164,0.100409,0.099930,0.100345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.083622,0.083623,0.083623,0.083624,0.083625,0.083627,0.083536,0.083446,0.083369,0.083279,...,0.064880,0.064884,0.064894,0.064949,0.064983,0.065103,0.065068,0.065253,0.065143,0.065412
0,0.082144,0.082141,0.082139,0.082137,0.082137,0.082138,0.082050,0.081962,0.081887,0.081798,...,0.136844,0.136603,0.136474,0.136278,0.136268,0.136343,0.136108,0.136211,0.135946,0.136133
0,0.075013,0.075006,0.074999,0.074995,0.074993,0.074993,0.074913,0.074834,0.074766,0.074686,...,0.104239,0.103798,0.103556,0.103174,0.103047,0.102962,0.102579,0.102466,0.102097,0.102016
0,0.067948,0.067949,0.067950,0.067951,0.067953,0.067956,0.067886,0.067816,0.067756,0.067687,...,0.059667,0.059469,0.059356,0.059186,0.059132,0.059105,0.058949,0.058913,0.058774,0.058758


In [15]:
def make_bins(wavs):
    """ Given a series of wavelength points, find the edges and widths
    of corresponding wavelength bins. """
    edges = np.zeros(wavs.shape[0]+1)
    widths = np.zeros(wavs.shape[0])
    edges[0] = wavs[0] - (wavs[1] - wavs[0])/2
    widths[-1] = (wavs[-1] - wavs[-2])
    edges[-1] = wavs[-1] + (wavs[-1] - wavs[-2])/2
    edges[1:-1] = (wavs[1:] + wavs[:-1])/2
    widths[:-1] = edges[1:-1] - edges[:-2]

    return edges, widths

In [17]:
def spec_resample(new_wavs, spec_wavs, spec_fluxes, spec_errs=None, fill=None, verbose=True):
    """
    Function for resampling spectra onto a new wavelength basis.
    """
    old_wavs = spec_wavs
    old_fluxes = spec_fluxes
    old_errs = spec_errs

    old_edges, old_widths = make_bins(old_wavs)
    new_edges, new_widths = make_bins(new_wavs)

    new_fluxes = np.zeros(old_fluxes[..., 0].shape + new_wavs.shape)

    if old_errs is not None:
        if old_errs.shape != old_fluxes.shape:
            raise ValueError("If specified, spec_errs must be the same shape as spec_fluxes.")
        else:
            new_errs = np.copy(new_fluxes)

    start = 0
    stop = 0
    warned = False

    for j in range(new_wavs.shape[0]):
        if (new_edges[j] < old_edges[0]) or (new_edges[j+1] > old_edges[-1]):
            new_fluxes[..., j] = fill

            if spec_errs is not None:
                new_errs[..., j] = fill

            if (j == 0 or j == new_wavs.shape[0]-1) and verbose and not warned:
                warned = True
                print("\nSpectres: new_wavs contains values outside the range in spec_wavs, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. \n")
            continue

        while old_edges[start+1] <= new_edges[j]:
            start += 1

        while old_edges[stop+1] < new_edges[j+1]:
            stop += 1

        if stop == start:
            new_fluxes[..., j] = old_fluxes[..., start]
            if old_errs is not None:
                new_errs[..., j] = old_errs[..., start]
        else:
            start_factor = ((old_edges[start+1] - new_edges[j]) / (old_edges[start+1] - old_edges[start]))
            end_factor = ((new_edges[j+1] - old_edges[stop]) / (old_edges[stop+1] - old_edges[stop]))

            old_widths[start] *= start_factor
            old_widths[stop] *= end_factor

            f_widths = old_widths[start:stop+1]*old_fluxes[..., start:stop+1]
            new_fluxes[..., j] = np.sum(f_widths, axis=-1)
            new_fluxes[..., j] /= np.sum(old_widths[start:stop+1])

            if old_errs is not None:
                e_wid = old_widths[start:stop+1]*old_errs[..., start:stop+1]
                new_errs[..., j] = np.sqrt(np.sum(e_wid**2, axis=-1))
                new_errs[..., j] /= np.sum(old_widths[start:stop+1])

            old_widths[start] /= start_factor
            old_widths[stop] /= end_factor

    if old_errs is not None:
        return new_fluxes, new_errs
    else:
        return new_fluxes

In [18]:
waves=list(range(400,2501))
waves=np.array(waves)
fwhm1=np.ones(2101)
fwhm1=np.array(fwhm1)
temp2=np.array(temp1)
temp2=temp2.T
temp3=np.array(temp1.iloc[0,:])

In [28]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from skimage import exposure
import builtins


# Reading header file
def read_header(file):
    f = builtins.open(file, 'r')
    lines = f.readlines()
    f.close()
    dict = {}
    # have_nonlowercase_param = False
    try:
        while lines:
            line = lines.pop(0)
            if line.find('=') == -1: continue
            if line[0] == ';': continue

            (key, sep, val) = line.partition('=')
            key = key.strip()
            if not key.islower():
                have_nonlowercase_param = True
                if not support_nonlowercase_params:
                    key = key.lower()
            val = val.strip()
            if val and val[0] == '{':
                str = val.strip()
                while str[-1] != '}':
                    line = lines.pop(0)
                    if line[0] == ';': continue

                    str += '\n' + line.strip()
                if key == 'description':
                    dict[key] = str.strip('{}').strip()
                else:
                    vals = str[1:-1].split(',')
                    for j in range(len(vals)):
                        vals[j] = vals[j].strip()
                    dict[key] = vals
            else:
                dict[key] = val
        return dict
    except:
        raise EnviHeaderParsingError()


import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from skimage import exposure
import builtins
# Reading raw data
Radiance_array = (rasterio.open('hsi_data/img_radiance.dat')).read()
Radiance_array=Radiance_array.transpose(1,2,0)
rows, cols, n_bands = Radiance_array.shape 
# Second axes
header = read_header('hsi_data/img_radiance.hdr')
wavelengths=np.asarray(header['wavelength'],dtype='float64')


In [29]:
fwhm1=np.ones(2101,dtype=np.uint32)
fwhm2=np.ones(272,dtype=np.uint32)
wavelengths1=wavelengths[1:,]
asd=[]
for i in range (5000):
    spec_resample_i, spec_errs_resample_i = spec_resample(wavelengths1, waves, temp2[:,i], fwhm1,fwhm2)
    asd.append(spec_resample_i)
prosail_ref=np.array(asd)

In [30]:
prosail_ref

array([[0.05097457, 0.05099969, 0.05098495, ..., 0.42454877, 0.42458369,
        0.42461868],
       [0.08094867, 0.08095471, 0.08088136, ..., 0.78430901, 0.78484801,
        0.78544464],
       [0.08692182, 0.0869444 , 0.08689252, ..., 0.80399823, 0.80434553,
        0.8047332 ],
       ...,
       [0.07500261, 0.07499365, 0.07491854, ..., 0.84324151, 0.84390233,
        0.84461307],
       [0.06794928, 0.06795238, 0.06789031, ..., 0.675135  , 0.67571657,
        0.67634235],
       [0.06824877, 0.06824891, 0.06817943, ..., 0.66180079, 0.66187783,
        0.66196361]])

In [76]:
bands2= np.array([443.9, 496.6,560, 664.5, 703.9,740.2, 782.5,835.1,864.8,945, 1613.7, 2202.4], dtype=np.uint32)
fwhm2=np.array([60,66,36,31,106,15,15,20,21,91,175,21,20,31],dtype=np.uint32)
asd=[]
for i in range (5000):
    spec_resample_i, spec_errs_resample_i = spec_resample(bands2, waves, temp2[:,i], fwhm1, fwhm2)
    asd.append(spec_resample_i)
    # asd=spec_resample.append
# e_spectra=spec_resample

In [77]:
asd1=np.array(asd)
import numpy as np
np.savetxt('s2.csv', asd1, delimiter=',')

In [2]:
import numpy as np
fwhm2=np.array([60,66,36,31,106,15,15,20,21,91,175,21,20,31],dtype=np.uint32)
fwhm2

array([ 60,  66,  36,  31, 106,  15,  15,  20,  21,  91, 175,  21,  20,
        31], dtype=uint32)